## Modules

In [1]:
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np

## Config

In [2]:
INCLUDE_IMPOSSIBLE = True
USE_MOD_ANSWER = False
OUTPUT_NAME = "w_noansw"

## Process data

In [3]:
pd.set_option('display.max_columns', None)
hun_data = pd.read_csv("data/dev_long_impossible_yes_no_arithmetic.csv", sep=';')

In [4]:
hun_data = hun_data.drop(hun_data.columns[0], axis=1) # drop nameless column
hun_data.head()

,id,user,context,title,section,is_impossible,question,end,start,answer,type,modanswer
0,1578,5,A török építészet formavilágának alapjai a nép...,Törökország,Építészet,True,Mikor épült az isztambuli Kék mecset?,0,0,NaN,NaN,NaN
1,3122,3,Puskás Ferenc apai ágon szegény sváb származás...,Puskás Ferenc (labdarúgó),"Gyermekkor, Kispesti AC",False,Miért nem a születése napján ünnepelte Puskás ...,256,0,Puskás Ferenc apai ágon szegény sváb származás...,[],NaN
2,2768,4,"Watson 2005-ben a Teen Vogue-ban, egy a kifeje...",Emma Watson,Modellkedés és divat,False,Melyik cég arca lett Emma Watson 2011 márciusá...,666,522,2011 januárjában Watson elnyerte az Elle Style...,[],NaN
3,1526,5,A kritikusok folyamatosan elemzik és vizsgáljá...,Lady Gaga,Zenei stílusa és témái,False,Miért változtatja folyton Gaga a zenei stílusát?,210,0,A kritikusok folyamatosan elemzik és vizsgáljá...,[],NaN
4,290,6,A hadihajók tüze után a szövetséges légierő ha...,Normandiai partraszállás,NaN,True,Melyik irányba tértek el a szövetséges bombázó...,0,0,NaN,NaN,NaN


In [5]:
hun_data.dtypes

id                int64
user              int64
context          object
title            object
section          object
is_impossible      bool
question         object
end               int64
start             int64
answer           object
type             object
modanswer        object
dtype: object

In [6]:
impossible_answer = "Nincs elegendő adat a kérdés megválaszolásához."

if INCLUDE_IMPOSSIBLE:
    prompt_prefix = ("Válaszold meg az alábbi kérdést a forrás alapján! "
                     "Ha a forrás alapján nem megválaszolható a kérdés, mondd hogy: "
                     f"{impossible_answer}")
else:
    prompt_prefix = "Válaszold meg az alábbi kérdést a forrás alapján!"

print(f"{prompt_prefix=}")
prompt_col = []
impossible_question_count = 0

with tqdm(total=len(hun_data.index), desc="Processing data") as pbar:
    for index, row in hun_data.loc[:].iterrows():

        if row["answer"] is np.nan:
            hun_data.loc[index, "is_impossible"] = True
        if hun_data.loc[index, "is_impossible"]:
            hun_data.loc[index, "answer"] = impossible_answer
            impossible_question_count = impossible_question_count + 1
        if USE_MOD_ANSWER and (row["modanswer"] is not np.nan):
            hun_data.loc[index, "answer"] = row["modanswer"]

        hun_data.loc[index, "answer"] = hun_data.loc[index, "answer"].strip(r" []'")

        text = (prompt_prefix
                + "\n### Forrás:\n" + str(row['context'])
                + "\n### Kérdés:\n" + str(row['question'])
                + "\n### Válasz:\n" + str(hun_data.loc[index, "answer"]))

        text = text.strip()
        prompt_col.append(text)
        pbar.update(1)
        
hun_data["text"] = prompt_col
print(f"{impossible_question_count=}")

prompt_prefix='Válaszold meg az alábbi kérdést a forrás alapján! Ha a forrás alapján nem megválaszolható a kérdés, mondd hogy: Nincs elegendő adat a kérdés megválaszolásához.'


Processing data: 100%|████████████████████████████████████████████████████████████| 4553/4553 [00:01<00:00, 3194.73it/s]

impossible_question_count=1335


In [7]:
print(f"{len(hun_data)=}")
if not INCLUDE_IMPOSSIBLE:
    hun_data = hun_data[hun_data.answer != impossible_answer]
    print(f"New {len(hun_data)=}")

len(hun_data)=4553


## Create train and test sets

In [8]:
print(f"{hun_data.shape=}")
hun_data.tail()

hun_data.shape=(4553, 13)


,id,user,context,title,section,is_impossible,question,end,start,answer,type,modanswer,text
4548,1491,5,Mozart ekkor végleg szakított a salzburgi érse...,Wolfgang Amadeus Mozart,NaN,False,Hol mutatták be elsőként a Don Giovanni című o...,1200,1138,1787-ben a Don Giovannit óriási sikerrel mutat...,[],NaN,Válaszold meg az alábbi kérdést a forrás alapj...
4549,3128,3,A világbajnokság után Puskás a Budapesti Honvé...,Puskás Ferenc (labdarúgó),Emigráció,True,A Honvéd vagy az Athletic Bilbao nyerte a brüs...,0,0,Nincs elegendő adat a kérdés megválaszolásához.,NaN,NaN,Válaszold meg az alábbi kérdést a forrás alapj...
4550,694,4,Az Hagia Szophia erőteljes szimbóluma a Görögk...,Hagia Szophia,Hagia Szophia – vallás és politika,False,Mire bizonyíték az Hagia Szophia Chris Spirou ...,772,588,Spirou szerint: „Az Hagia Szophia a legnagyobb...,[],NaN,Válaszold meg az alábbi kérdést a forrás alapj...
4551,271,6,"A lap székhelyéül Münchent választották, ugyan...",Vlagyimir Iljics Lenin,Az első emigráció,True,"Mi volt az eredeti orosz címe Lenin ""Mi a teen...",0,0,Nincs elegendő adat a kérdés megválaszolásához.,NaN,NaN,Válaszold meg az alábbi kérdést a forrás alapj...
4552,1901,5,"A T m. aterrimus Magyarországon költ, míg déle...",Fekete rigó,Alfajai,True,Hány nap múlva kelnek ki a tojásból a fekete r...,0,0,Nincs elegendő adat a kérdés megválaszolásához.,NaN,NaN,Válaszold meg az alábbi kérdést a forrás alapj...


In [9]:
hun_data_train, hun_data_temp = train_test_split(hun_data, test_size=0.3, random_state=42)
hun_data_eval, hun_data_test = train_test_split(hun_data_temp, test_size=0.5, random_state=42)

In [10]:
print("hun_data_train shape:", hun_data_train.shape)
print("hun_data_test shape:", hun_data_test.shape)
print("hun_data_eval shape:", hun_data_eval.shape)

hun_data_train shape: (3187, 13)
hun_data_test shape: (683, 13)
hun_data_eval shape: (683, 13)


In [11]:
hun_data_train_q_c_a_t = hun_data_train[['question', 'context', 'answer', 'text']]
hun_data_test_q_c_a_t = hun_data_test[['question', 'context', 'answer', 'text']]
hun_data_eval_q_c_a_t = hun_data_eval[['question', 'context', 'answer', 'text']]

In [12]:
hun_data_train_q_c_a_t.head()

,question,context,answer,text
1356,Milyen nagy fesztiválokat tartanak manapság In...,A fesztivált 1992 óta az indiai Turisztikai Mi...,Nincs elegendő adat a kérdés megválaszolásához.,Válaszold meg az alábbi kérdést a forrás alapj...
2823,Milyen rangban volt az ügyeletes a Reichswehr ...,Az idő előrehaladásával párhuzamosan egyre job...,Nincs elegendő adat a kérdés megválaszolásához.,Válaszold meg az alábbi kérdést a forrás alapj...
1456,Miért az akusztikus hallucinációk fordulnak el...,"Érzékelés: általánosságban jellemző, hogy a re...",Nincs elegendő adat a kérdés megválaszolásához.,Válaszold meg az alábbi kérdést a forrás alapj...
396,"Mi kellene ahhoz Visy Zsolt szerint, hogy a Se...","2008-ban Visy Zsolt régész, egyetemi tanár öss...","2008-ban Visy Zsolt régész, egyetemi tanár öss...",Válaszold meg az alábbi kérdést a forrás alapj...
1064,Milyen módszerrel épült a Vaskapu menti út Tib...,Az ókorban a Vaskapu oly mértékben elválasztot...,I. sz. 33–34 során egy 210 méter hosszú utat é...,Válaszold meg az alábbi kérdést a forrás alapj...


In [13]:
hun_data_test_q_c_a_t.head()

,question,context,answer,text
4518,Miért Rommelt nevezte ki Hitler a francia védm...,1943 novemberében Hitler Erwin Rommelt nevezte...,Nincs elegendő adat a kérdés megválaszolásához.,Válaszold meg az alábbi kérdést a forrás alapj...
3455,Mikor veszítette el édesanyját Johann Sebastia...,A gyermeket hét-nyolc éves korában az 1692–169...,A korabeli feljegyzések szerint a gyermek Bach...,Válaszold meg az alábbi kérdést a forrás alapj...
1345,Hány lakóháazt építettek a Wekerle-telepre?,A telepes lakásépítés a századfordulón egyre n...,A kor legismertebb példája a budapesti Wekerle...,Válaszold meg az alábbi kérdést a forrás alapj...
540,Kitől kell függetlennek lennie az OEKO-TEX STe...,a szennyvíz és a szennyezett levegő tisztításá...,Nincs elegendő adat a kérdés megválaszolásához.,Válaszold meg az alábbi kérdést a forrás alapj...
564,Ilja Petrovics Csajkovszkijnak és Alekszandra...,1833. október 13-án Ilja Petrovics újból megnő...,"Házasságukból hét gyermek született, de első l...",Válaszold meg az alábbi kérdést a forrás alapj...


In [14]:
hun_data_eval_q_c_a_t.head()

,question,context,answer,text
3958,Ki írt először a zsidók prágai megtelepedéséről?,"Alapításáról több legenda is kering, egyik sze...",Ekkor telepedtek meg itt a város későbbi életé...,Válaszold meg az alábbi kérdést a forrás alapj...
3262,Mikor alapították a Magyar Mérnök és Építész E...,A magyar szecesszió meghatározó építészeinek i...,Az 1867-ben alakult Magyar Mérnök és Építész E...,Válaszold meg az alábbi kérdést a forrás alapj...
1874,Miért nem lehet legálisan értékesíteni a Seuso...,A kincsek eredetének kiderítésére 1993-ban Mag...,Nincs elegendő adat a kérdés megválaszolásához.,Válaszold meg az alábbi kérdést a forrás alapj...
2285,Hogyan alakult az olasz-német viszony az 1930-...,Az első ilyen diktátor a 20. század történetéb...,A diktátor kezdetben ellenségesen viszonyult a...,Válaszold meg az alábbi kérdést a forrás alapj...
3978,Mekkora veszteséggel számoltak az angolok a no...,Az akcióban részt vevő flottaegységek parancsn...,Nincs elegendő adat a kérdés megválaszolásához.,Válaszold meg az alábbi kérdést a forrás alapj...


In [15]:
hun_data_train_q_c_a_t.to_csv("data/train_w_noansw.csv", index=False, sep=";")
hun_data_test_q_c_a_t.to_csv("data/test_w_noansw.csv", index=False, sep=";")
hun_data_eval_q_c_a_t.to_csv("data/eval_w_noansw.csv", index=False, sep=";")